In [1]:
from awrams.simulation.server import Server
from awrams.models import awral

from awrams.utils import extents
from awrams.utils import datetools

## AusCover data can be downloaded from thredds server:
http://rs-data1-mel.csiro.au/thredds/fileServer/bawap/

In [2]:
def change_forcing(imap):
    from awrams.utils.nodegraph.nodes import forcing_from_ncfiles
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'
    
    imap.mapping['precip_f'] = forcing_from_ncfiles(data_path,'bom-rain_day-*','rain_day')
    imap.mapping['tmax_f']   = forcing_from_ncfiles(data_path,'bom-tmax_day-*','tmax_day')
    imap.mapping['tmin_f']   = forcing_from_ncfiles(data_path,'bom-tmin_day-*','tmin_day')
    imap.mapping['solar_f']  = forcing_from_ncfiles(data_path,'bom-rad_day-*','rad_day')


In [3]:
imap = awral.get_default_mapping()
change_forcing(imap)

### create model output map and add nodes for saving some model outputs to netcdf files

In [4]:
def build_output_mapping():
    from awrams.utils.nodegraph import nodes,graph
    from awrams.models.awral.template import DEFAULT_TEMPLATE

    ### populate output map with all model outputs
    output_map = awral.get_output_nodes(DEFAULT_TEMPLATE)

    outpath = './_results/'

    output_map.mapping['s0_ncsave'] = nodes.write_to_annual_ncfile(outpath,'s0')
    output_map.mapping['ss_ncsave'] = nodes.write_to_annual_ncfile(outpath,'ss')
    output_map.mapping['sd_ncsave'] = nodes.write_to_annual_ncfile(outpath,'sd')
    output_map.mapping['qtot_ncsave'] = nodes.write_to_annual_ncfile(outpath,'qtot')
    output_map.mapping['etot_ncsave'] = nodes.write_to_annual_ncfile(outpath,'etot')

    return graph.OutputGraph(output_map.mapping)


In [5]:
omap = build_output_mapping()

In [6]:
period = datetools.dates('dec 2010 - jan 2011')
extent = extents.from_boundary_offset(200,200,250,250)

In [7]:
period

DatetimeIndex(['2010-12-01', '2010-12-02', '2010-12-03', '2010-12-04',
               '2010-12-05', '2010-12-06', '2010-12-07', '2010-12-08',
               '2010-12-09', '2010-12-10', '2010-12-11', '2010-12-12',
               '2010-12-13', '2010-12-14', '2010-12-15', '2010-12-16',
               '2010-12-17', '2010-12-18', '2010-12-19', '2010-12-20',
               '2010-12-21', '2010-12-22', '2010-12-23', '2010-12-24',
               '2010-12-25', '2010-12-26', '2010-12-27', '2010-12-28',
               '2010-12-29', '2010-12-30', '2010-12-31', '2011-01-01',
               '2011-01-02', '2011-01-03', '2011-01-04', '2011-01-05',
               '2011-01-06', '2011-01-07', '2011-01-08', '2011-01-09',
               '2011-01-10', '2011-01-11', '2011-01-12', '2011-01-13',
               '2011-01-14', '2011-01-15', '2011-01-16', '2011-01-17',
               '2011-01-18', '2011-01-19', '2011-01-20', '2011-01-21',
               '2011-01-22', '2011-01-23', '2011-01-24', '2011-01-25',
      

In [8]:
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:25:17,394 INFO Getting I/O dataspecs...
2016-10-13 04:25:19,358 INFO Initialising output files...
2016-10-13 04:25:19,550 INFO Building buffers...
2016-10-13 04:25:20,980 INFO Running simulation...


worker pid: 10127
worker pid: 10128
worker pid: 10131
worker pid: 10139


2016-10-13 04:25:22,852 INFO completed 25.00%
2016-10-13 04:25:21,029 INFO writer pid: 10153
2016-10-13 04:25:21,030 INFO reader pid: 10152
2016-10-13 04:25:22,944 INFO completed 50.00%
2016-10-13 04:25:23,048 INFO completed 75.00%
2016-10-13 04:25:23,093 INFO completed 100.00%
2016-10-13 04:25:23,115 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:25:23,202 INFO completed 25.00%
2016-10-13 04:25:23,330 INFO completed 50.00%
2016-10-13 04:25:23,386 INFO completed 75.00%
2016-10-13 04:25:23,463 INFO completed 100.00%
2016-10-13 04:25:23,474 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:25:25,058 INFO elapsed time: 7.68


reader pid: 10152


### initialise states from dict mapping of numpy arrays

In [9]:
def initial_states_from_dict(imap,period,extent):
    from awrams.utils.io.data_mapping import SplitFileManager
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'

    node_names = {'mleaf_dr': 'init_mleaf_hrudr',
                  'mleaf_sr': 'init_mleaf_hrusr',
                  's0_dr': 'init_s0_hrudr',
                  's0_sr': 'init_s0_hrusr',
                  'ss_dr': 'init_ss_hrudr',
                  'ss_sr': 'init_ss_hrusr',
                  'sd_dr': 'init_sd_hrudr',
                  'sd_sr': 'init_sd_hrusr',
                  'sg_bal': 'init_sg',
                  'sr_bal': 'init_sr'}

    data_map = {}
    period = [period[0] - 1]
    
    for k in node_names:
        sfm = SplitFileManager.open_existing(data_path,k+'*nc',k)
        data_map[node_names[k]] = sfm.get_data(period,extent)

    nodes.init_states_from_dict(imap,data_map,extent)


In [10]:
initial_states_from_dict(imap,period,extent)
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:25:25,399 INFO Getting I/O dataspecs...
2016-10-13 04:25:27,405 INFO Initialising output files...
2016-10-13 04:25:27,544 INFO Building buffers...
2016-10-13 04:25:27,665 INFO Running simulation...


worker pid: 10223
worker pid: 10224
worker pid: 10229
worker pid: 10238


2016-10-13 04:25:29,557 INFO completed 25.00%
2016-10-13 04:25:27,705 INFO writer pid: 10251
2016-10-13 04:25:27,706 INFO reader pid: 10248
2016-10-13 04:25:29,652 INFO completed 50.00%
2016-10-13 04:25:29,728 INFO completed 75.00%
2016-10-13 04:25:29,826 INFO completed 100.00%
2016-10-13 04:25:29,847 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:25:29,913 INFO completed 25.00%
2016-10-13 04:25:30,006 INFO completed 50.00%
2016-10-13 04:25:30,113 INFO completed 75.00%
2016-10-13 04:25:30,179 INFO completed 100.00%
2016-10-13 04:25:30,186 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:25:30,809 INFO elapsed time: 5.43


reader pid: 10248


### initialise states from netcdf files

In [11]:
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'

    mapping = imap.mapping

    mapping['init_sr'] = nodes.init_state_from_ncfile(data_path,'sr_bal*','sr_bal')
    mapping['init_sg'] = nodes.init_state_from_ncfile(data_path,'sg_bal*','sg_bal')

    HRU = {'_hrusr':'_sr','_hrudr':'_dr'}
    for hru in ('_hrusr','_hrudr'):
        for state in ["s0","ss","sd",'mleaf']:
            mapping['init_'+state+hru] = nodes.init_state_from_ncfile(data_path,state+HRU[hru]+'*',state+HRU[hru])


In [12]:
initial_states_from_dict(imap,period,extent)
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:25:31,139 INFO Getting I/O dataspecs...
2016-10-13 04:25:33,112 INFO Initialising output files...
2016-10-13 04:25:33,249 INFO Building buffers...
2016-10-13 04:25:33,370 INFO Running simulation...


worker pid: 10318
worker pid: 10319
worker pid: 10324
worker pid: 10333


2016-10-13 04:25:35,222 INFO completed 25.00%
2016-10-13 04:25:33,413 INFO writer pid: 10346
2016-10-13 04:25:33,414 INFO reader pid: 10343
2016-10-13 04:25:35,335 INFO completed 50.00%
2016-10-13 04:25:35,392 INFO completed 75.00%
2016-10-13 04:25:35,489 INFO completed 100.00%
2016-10-13 04:25:35,511 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:25:35,577 INFO completed 25.00%
2016-10-13 04:25:35,707 INFO completed 50.00%
2016-10-13 04:25:35,762 INFO completed 75.00%
2016-10-13 04:25:35,858 INFO completed 100.00%
2016-10-13 04:25:35,867 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:25:38,431 INFO elapsed time: 7.31


reader pid: 10343


### fill gaps in forcing data with climatology

In [13]:
def insert_climatology(imap):
    from awrams.utils.nodegraph import nodes
    from os.path import join
    from os import getcwd

    data_path = '../../test_data/simulation/'

    imap.mapping['precip_f'] = nodes.forcing_gap_filler(data_path,'rain_day*','rain_day',join(data_path,'climatology_daily_rain_day.nc'))
    imap.mapping['tmax_f'] = nodes.forcing_gap_filler(data_path,'temp_max*','temp_max_day',join(data_path,'climatology_daily_temp_max_day.nc'))
    imap.mapping['tmin_f'] = nodes.forcing_gap_filler(data_path,'temp_min*','temp_min_day',join(data_path,'climatology_daily_temp_min_day.nc'))
    imap.mapping['solar_f'] = nodes.forcing_gap_filler(data_path,'solar*','solar_exposure_day',join(data_path,'climatology_daily_solar_exposure_day.nc'))
    

In [14]:
insert_climatology(imap)
sim = Server(awral)
sim.run(imap,omap,period,extent)

2016-10-13 04:25:38,482 INFO Getting I/O dataspecs...
2016-10-13 04:25:39,190 INFO Initialising output files...
2016-10-13 04:25:39,341 INFO Building buffers...
2016-10-13 04:25:39,462 INFO Running simulation...


worker pid: 10413
worker pid: 10414
worker pid: 10419
worker pid: 10428


2016-10-13 04:25:40,670 INFO completed 25.00%
2016-10-13 04:25:39,511 INFO reader pid: 10438
2016-10-13 04:25:39,510 INFO writer pid: 10441
2016-10-13 04:25:40,764 INFO completed 50.00%
2016-10-13 04:25:40,870 INFO completed 75.00%
2016-10-13 04:25:40,970 INFO completed 100.00%
2016-10-13 04:25:40,993 INFO Completed period December 2010 - 1 of 2
2016-10-13 04:25:41,060 INFO completed 25.00%
2016-10-13 04:25:41,153 INFO completed 50.00%
2016-10-13 04:25:41,243 INFO completed 75.00%
2016-10-13 04:25:41,316 INFO completed 100.00%
2016-10-13 04:25:41,317 INFO Completed period January 2011 - 2 of 2
2016-10-13 04:25:42,932 INFO elapsed time: 4.47


reader pid: 10438
